In [1]:
certificate_folder = "C:/Users/Tomonobu/Documents/home/Keyence/ラズパイIoT勉強会/証明書_TestVirtualMachine/"

In [2]:
filepath_route_ca    = certificate_folder + "VeriSign-Class 3-Public-Primary-Certification-Authority-G5.pem"
filepath_private_key = certificate_folder + "0f19545793-private.pem.key"
filepath_certificate = certificate_folder + "0f19545793-certificate.pem.crt"


In [3]:
aws_deviceid = "test_vm_ubuntu2"
endpoint = "a3i29lc4oae1iw.iot.ap-northeast-1.amazonaws.com"

from enum import Enum
class Signal_State(Enum):
    R_STOP = 1
    YY_WARNING = 2
    Y_CAUTION = 3
    YG_DECEL = 4
    Y_GO = 5

In [4]:
# Custom Shadow callback
def customShadowCallback_Update(payload, responseStatus, token):
    # payload is a JSON string ready to be parsed using json.loads(...)
    # in both Py2.x and Py3.x
    if responseStatus == "timeout":
        print("Update request " + token + " time out!")
    if responseStatus == "accepted":
        payloadDict = json.loads(payload)
    if responseStatus == "rejected":
        print("Update request " + token + " rejected!")

def customShadowCallback_Delete(payload, responseStatus, token):
    if responseStatus == "timeout":
        print("Delete request " + token + " time out!")
    if responseStatus == "accepted":
        print("~~~~~~~~~~~~~~~~~~~~~~~")
        print("Delete request with token: " + token + " accepted!")
        print("~~~~~~~~~~~~~~~~~~~~~~~\n\n")
    if responseStatus == "rejected":
        print("Delete request " + token + " rejected!")


In [5]:
Ctrl_Values = {
    'Ctrl_Val0':0, 
    'Ctrl_Val1':1,
    'Ctrl_Val2':2,
    'Ctrl_Val3':3}

State_Values = {
    'State_Val0':0, 
    'State_Val1':1,
    'State_Val2':2,
    'State_Val3':3}    

# Custom Shadow callback
def customShadowCallback_Delta(payload, responseStatus, token):
    global Ctrl_Values
    
    # payload is a JSON string ready to be parsed using json.loads(...)
    # in both Py2.x and Py3.x
    print(responseStatus)
    payloadDict = json.loads(payload)
    print(payload)
    print("++++++++DELTA++++++++++")
    print("version: " + str(payloadDict["version"]))
    print("+++++++++++++++++++++++\n\n")

    for k, v in payloadDict["state"].items():
        Ctrl_Values[k] = int(v)
    

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

import logging
import time
import json
import argparse

myAWSIoTMQTTShadowClient = AWSIoTMQTTShadowClient(aws_deviceid)
myAWSIoTMQTTShadowClient.configureEndpoint(endpoint, 8883)
myAWSIoTMQTTShadowClient.configureCredentials(filepath_route_ca, filepath_private_key, filepath_certificate)

# AWSIoTMQTTShadowClient configuration
myAWSIoTMQTTShadowClient.configureAutoReconnectBackoffTime(1, 32, 20)
myAWSIoTMQTTShadowClient.configureConnectDisconnectTimeout(10)  # 10 sec
myAWSIoTMQTTShadowClient.configureMQTTOperationTimeout(5)  # 5 sec

# Connect to AWS IoT
myAWSIoTMQTTShadowClient.connect()

# MQTTClient = myAWSIoTMQTTShadowClient.getMQTTConnection()
# MQTTClient.configureOfflinePublishQueueing(yourQueueSize, yourDropBehavior)

# Create a deviceShadow with persistent subscription
deviceShadowHandler = myAWSIoTMQTTShadowClient.createShadowHandlerWithName("test_vm_ubuntu_2", True)

# Delete shadow JSON doc
deviceShadowHandler.shadowDelete(customShadowCallback_Delete, 5)

# Create a deviceShadow with persistent subscription
# deviceShadowHandler = myAWSIoTMQTTShadowClient.createShadowHandlerWithName("MyBot", True)

# Listen on deltas
deviceShadowHandler.shadowRegisterDeltaCallback(customShadowCallback_Delta)

# Update shadow desired init state
JSONPayload = '{"state":{"desired":{"Ctrl_Val0":' + str(Ctrl_Values['Ctrl_Val0']) + ',' \
                                    + '"Ctrl_Val1":' + str(Ctrl_Values['Ctrl_Val1']) + ',' \
                                    + '"Ctrl_Val2":' + str(Ctrl_Values['Ctrl_Val2']) + ',' \
                                    + '"Ctrl_Val3":' + str(Ctrl_Values['Ctrl_Val3']) \
                + '}}}'
deviceShadowHandler.shadowUpdate(JSONPayload, customShadowCallback_Update, 5)


# Update shadow in a loop
loopCount = 0
while True:
    JSONPayload = '{"state":{"reported":{"Ctrl_Val0":' + str(Ctrl_Values['Ctrl_Val0']) + ',' \
                                        + '"Ctrl_Val1":' + str(Ctrl_Values['Ctrl_Val1']) + ',' \
                                        + '"Ctrl_Val2":' + str(Ctrl_Values['Ctrl_Val2']) + ',' \
                                        + '"Ctrl_Val3":' + str(Ctrl_Values['Ctrl_Val3']) + ',' \
                                        + '"State_Val0":' + str(State_Values['State_Val0']) + ',' \
                                        + '"State_Val1":' + str(State_Values['State_Val1']) + ',' \
                                        + '"State_Val2":' + str(State_Values['State_Val2']) + ',' \
                                        + '"State_Val3":' + str(State_Values['State_Val3']) \
                    + '}}}'
    print(JSONPayload)
    deviceShadowHandler.shadowUpdate(JSONPayload, customShadowCallback_Update, 5)
    State_Values['State_Val0'] += 1
    State_Values['State_Val2'] += 2
    time.sleep(1)


~~~~~~~~~~~~~~~~~~~~~~~
Delete request with token: e1e5a395-04d9-43f5-9198-3a1035f9d26c accepted!
~~~~~~~~~~~~~~~~~~~~~~~


{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":0,"State_Val1":1,"State_Val2":2,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":1,"State_Val1":1,"State_Val2":4,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":2,"State_Val1":1,"State_Val2":6,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3,"State_Val1":1,"State_Val2":8,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":4,"State_Val1":1,"State_Val2":10,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":5,"State_Val1":1,"State_Val2":12,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":57,"State_Val1":1,"State_Val2":116,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":58,"State_Val1":1,"State_Val2":118,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":59,"State_Val1":1,"State_Val2":120,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":60,"State_Val1":1,"State_Val2":122,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":61,"State_Val1":1,"State_Val2":124,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":62,"State_Val1":1,"State_Val2":126,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":63,"State_Val1":1,"State_Val2":128,"State_Val

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":114,"State_Val1":1,"State_Val2":230,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":115,"State_Val1":1,"State_Val2":232,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":116,"State_Val1":1,"State_Val2":234,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":117,"State_Val1":1,"State_Val2":236,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":118,"State_Val1":1,"State_Val2":238,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":119,"State_Val1":1,"State_Val2":240,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":120,"State_Val1":1,"State_Val2":242,"St

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":171,"State_Val1":1,"State_Val2":344,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":172,"State_Val1":1,"State_Val2":346,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":173,"State_Val1":1,"State_Val2":348,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":174,"State_Val1":1,"State_Val2":350,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":175,"State_Val1":1,"State_Val2":352,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":176,"State_Val1":1,"State_Val2":354,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":177,"State_Val1":1,"State_Val2":356,"St

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":228,"State_Val1":1,"State_Val2":458,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":229,"State_Val1":1,"State_Val2":460,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":230,"State_Val1":1,"State_Val2":462,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":231,"State_Val1":1,"State_Val2":464,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":232,"State_Val1":1,"State_Val2":466,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":233,"State_Val1":1,"State_Val2":468,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":234,"State_Val1":1,"State_Val2":470,"St

...

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3601,"State_Val1":1,"State_Val2":7204,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3602,"State_Val1":1,"State_Val2":7206,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3603,"State_Val1":1,"State_Val2":7208,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3604,"State_Val1":1,"State_Val2":7210,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3605,"State_Val1":1,"State_Val2":7212,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3606,"State_Val1":1,"State_Val2":7214,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3607,"State_Val1":1,"State_

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3657,"State_Val1":1,"State_Val2":7316,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3658,"State_Val1":1,"State_Val2":7318,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3659,"State_Val1":1,"State_Val2":7320,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3660,"State_Val1":1,"State_Val2":7322,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3661,"State_Val1":1,"State_Val2":7324,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3662,"State_Val1":1,"State_Val2":7326,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3663,"State_Val1":1,"State_

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3713,"State_Val1":1,"State_Val2":7428,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3714,"State_Val1":1,"State_Val2":7430,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3715,"State_Val1":1,"State_Val2":7432,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3716,"State_Val1":1,"State_Val2":7434,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3717,"State_Val1":1,"State_Val2":7436,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3718,"State_Val1":1,"State_Val2":7438,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3719,"State_Val1":1,"State_

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3769,"State_Val1":1,"State_Val2":7540,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3770,"State_Val1":1,"State_Val2":7542,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3771,"State_Val1":1,"State_Val2":7544,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3772,"State_Val1":1,"State_Val2":7546,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3773,"State_Val1":1,"State_Val2":7548,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3774,"State_Val1":1,"State_Val2":7550,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3775,"State_Val1":1,"State_

{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3825,"State_Val1":1,"State_Val2":7652,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3826,"State_Val1":1,"State_Val2":7654,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3827,"State_Val1":1,"State_Val2":7656,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3828,"State_Val1":1,"State_Val2":7658,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3829,"State_Val1":1,"State_Val2":7660,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3830,"State_Val1":1,"State_Val2":7662,"State_Val3":3}}}
{"state":{"reported":{"Ctrl_Val0":0,"Ctrl_Val1":1,"Ctrl_Val2":2,"Ctrl_Val3":3,"State_Val0":3831,"State_Val1":1,"State_

In [ ]:
'{"state":{"reported":{"Ctrl_Val0":' + str(State_Ctrl_Val0) + ',' \
                                    + '"Ctrl_Val1":' + str(State_Ctrl_Val1) + ',' \
                                    + '"Ctrl_Val2":' + str(State_Ctrl_Val2) + ',' \
                                    + '"Ctrl_Val3":' + str(State_Ctrl_Val3) + \
                + '}}}'


In [8]:
 str(State_Ctrl_Val1)

'1'

In [9]:
str(State_Ctrl_Val3)

'3'